In [17]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import math
import copy as cp
import matplotlib.pyplot as plt 
import xgboost as xgb
# https://blog.csdn.net/Jerr__y/article/details/79005842
# https://github.com/neal668/LightGBM-GBDT-LR/blob/master/GBFT%2BLR_simple.py

In [54]:
df1=pd.read_csv("../data/first_round_training_data.csv")
df2=pd.read_csv("../data/first_round_testing_data.csv")

train_x=np.zeros([6000,10],dtype=np.float32)
train_y=np.zeros([6000],dtype=np.int32)

test_id=np.zeros([6000],dtype=np.int32)
test_x=np.zeros([6000,10],dtype=np.float32)

cls2int={"Excellent":0,"Good":1,"Pass":2,"Fail":3}

for i in range(1,11):
    par="Parameter"+str(i)
    tmp=df1[par]
    for j in range(6000):
        train_x[j,i-1]=tmp[j]
        cls=cls2int[df1["Quality_label"][j]]
        train_y[j]=cls
for i in range(1,11):
    par="Parameter"+str(i)
    tmp=df2[par]
    for j in range(6000):
        test_x[j,i-1]=tmp[j]
        ID=int(df2["Group"][j])
        test_id[j]=ID

X_train, X_validation, y_train, y_validation = train_test_split(\
                    train_x, train_y, test_size=0.3 , random_state=1234)

In [51]:
model = xgb.XGBClassifier(
    booster = 'gbtree',
    objective = 'multi:softmax',
    num_class = 4,
    gamma=0.1,
    max_depth=6,
    n_estimators=500,
    learning_rate=0.01,
    reg_alpha=5,
    reg_lambda=2,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=3,
    scale_pos_weight=1.6, 
    seed=1000,
    silent=False,
    n_thread = 4
    #eval_metric='error'
    )

model.fit(X_train, y_train, 
          eval_set=[(X_train, y_train), (X_validation, y_validation)],
          #eval_metric=['merror', 'mlogloss'],
          verbose=True,
          early_stopping_rounds=10
         )

[0]	validation_0-merror:0.454286	validation_1-merror:0.522778
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 10 rounds.
[1]	validation_0-merror:0.454048	validation_1-merror:0.515
[2]	validation_0-merror:0.442857	validation_1-merror:0.510556
[3]	validation_0-merror:0.445	validation_1-merror:0.51
[4]	validation_0-merror:0.435476	validation_1-merror:0.505
[5]	validation_0-merror:0.430238	validation_1-merror:0.506111
[6]	validation_0-merror:0.436429	validation_1-merror:0.502778
[7]	validation_0-merror:0.434286	validation_1-merror:0.503889
[8]	validation_0-merror:0.435952	validation_1-merror:0.502222
[9]	validation_0-merror:0.431429	validation_1-merror:0.503333
[10]	validation_0-merror:0.434524	validation_1-merror:0.503889
[11]	validation_0-merror:0.433095	validation_1-merror:0.5
[12]	validation_0-merror:0.431905	validation_1-merror:0.498889
[13]	validation_0-merror:0.435	validation_1-me

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.1,
              learning_rate=0.01, max_delta_step=0, max_depth=6,
              min_child_weight=3, missing=None, n_estimators=500, n_jobs=1,
              n_thread=4, nthread=None, num_class=4, objective='multi:softprob',
              random_state=0, reg_alpha=5, reg_lambda=2, scale_pos_weight=1.6,
              seed=1000, silent=False, subsample=0.8, verbosity=1)

In [52]:
# 对应组别所有工件质检结果为优的比率

import datetime
name = str(datetime.datetime.now().month) + str(datetime.datetime.now().day)

ans = model.predict(test_x)
df_new = pd.concat([pd.DataFrame(test_id), pd.DataFrame(ans)], axis=1)
df_new.columns = ['Group', 'type']

df_result = df_new.groupby(['Group', 'type'])['type'].count()/50
df_result = df_result.unstack()
df_result.fillna(0, inplace=True)
df_result.columns = ['Excellent ratio','Good ratio','Pass ratio','Fail ratio']
print(df_result)
df_result.to_csv("../data/result_xbt_1_%s.csv"% name)

       Excellent ratio  Good ratio  Pass ratio  Fail ratio
Group                                                     
0                 0.18        0.60        0.12        0.10
1                 0.12        0.28        0.46        0.14
2                 0.32        0.50        0.18        0.00
3                 0.32        0.30        0.34        0.04
4                 0.04        0.64        0.28        0.04
...                ...         ...         ...         ...
115               0.10        0.34        0.42        0.14
116               0.00        0.04        0.92        0.04
117               0.10        0.44        0.34        0.12
118               0.00        0.00        1.00        0.00
119               0.16        0.32        0.40        0.12

[120 rows x 4 columns]


In [53]:
ans = model.predict_proba(test_x)

df_new = pd.concat([pd.DataFrame(test_id), pd.DataFrame(ans)], axis=1)
df_new.columns = ['Group', 'Excellent ratio','Good ratio','Pass ratio','Fail ratio']

df_result = df_new.groupby('Group')['Excellent ratio','Good ratio','Pass ratio','Fail ratio'].mean()
df_result.to_csv("../data/result_xgb_2_%s.csv"% name)
print(df_result.iloc[:10,:])

       Excellent ratio  Good ratio  Pass ratio  Fail ratio
Group                                                     
0             0.247509    0.255190    0.251329    0.245972
1             0.245520    0.250177    0.260594    0.243710
2             0.252477    0.253185    0.252864    0.241474
3             0.249512    0.252631    0.258922    0.238934
4             0.245189    0.257202    0.250927    0.246682
5             0.245440    0.249543    0.262531    0.242486
6             0.245689    0.251592    0.258681    0.244037
7             0.243067    0.250165    0.266398    0.240369
8             0.240181    0.239581    0.284551    0.235686
9             0.244185    0.250348    0.262509    0.242957
